# Imminent ICU Admission Classifier with MLH Notes only using Logistic Regression

## Imports & Inits

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../../')

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

import pickle
import scipy

import pandas as pd
import numpy as np
np.set_printoptions(precision=4)

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
%matplotlib inline

from pathlib import Path

from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from scikitplot.metrics import *

from utils.data_utils import set_group_splits
from utils.metrics import BinaryAvgMetrics, get_best_model
from utils.plots import *

In [2]:
from args import args
vars(args)

{'path': PosixPath('../data'),
 'workdir': PosixPath('../data/workdir'),
 'figdir': PosixPath('../data/figures'),
 'dataset_csv': PosixPath('../data/unstructured_proc.csv'),
 'threshold': 0.31}

In [3]:
seed = 643
notes_df = pd.read_csv(args.dataset_csv, usecols=['hadm_id', 'note', 'imi_adm_label'])
notes_df = notes_df[notes_df['imi_adm_label'] != -1].reset_index(drop=True)
notes_df.shape

(116400, 3)

In [4]:
save = False

## Functions

In [5]:
def compute_cloud(scores, idx2tok, n_tokens=50, min_len=4):
  sorted_idxs = np.argsort(scores)  
  p = re.compile('^[a-z\s]+$')
  tokens = []
  trim_scores = []

  for idx in sorted_idxs:
    token = idx2tok[idx].lower()
    if len(token) > min_len and p.match(token):
      tokens.append(token)
      trim_scores.append(scores[idx] * -1)
      
  pos_dict = {k: v for k, v in zip(tokens[:n_tokens], trim_scores[:n_tokens])}
  neg_dict = {k: v for k, v in zip(tokens[-n_tokens:], trim_scores[-n_tokens:])}
  
  pos_cloud = WordCloud(width=400, height=400, background_color='white', max_font_size=60, relative_scaling=0.5).generate_from_frequencies(pos_dict)
  neg_cloud = WordCloud(width=400, height=400, background_color='white', max_font_size=60, relative_scaling=0.5).generate_from_frequencies(neg_dict)
  
  return pos_cloud, neg_cloud

## Single Model Development

In [6]:
df = set_group_splits(notes_df.copy(), group_col='hadm_id', seed=seed)

train_df = df[df['split'] == 'train'][['note', 'imi_adm_label']]
test_df = df[df['split'] == 'test'][['note', 'imi_adm_label']]

g = train_df.groupby(['imi_adm_label']).size().to_numpy()
print(f"Prevalence of positive class in training set:{(g[1]/g.sum())*100:0.1f}%")
g = test_df.groupby(['imi_adm_label']).size().to_numpy()
print(f"Prevalence of positive class in test set:{(g[1]/g.sum())*100:0.1f}%")

Prevalence of positive class in training set:3.3%
Prevalence of positive class in test set:3.6%


In [ ]:
# vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features=60_000)
# x_train = vectorizer.fit_transform(train_df['note'])
# x_test = vectorizer.transform(test_df['note'])

# with open(args.workdir/f'vectordir/default_bi_643.pkl', 'wb') as f:
#   pickle.dump(vectorizer, f)
#   pickle.dump(x_train, f)
#   pickle.dump(x_test, f)

with open(args.workdir/f'vectordir/default_bi_643.pkl', 'rb') as f:
  vectorizer = pickle.load(f)
  x_train = pickle.load(f)
  x_test = pickle.load(f)
  
y_train,y_test = train_df['imi_adm_label'], test_df['imi_adm_label']
x_train.shape, x_test.shape, y_train.shape, y_test.shape

In [ ]:
clf = LogisticRegression(class_weight='balanced')
clf.fit(x_train, y_train)

prob = clf.predict_proba(x_test)
pos_prob = prob[:, 1]

labels = ['Delayed', 'Imminent']
label_test = [labels[i] for i in y_test]

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
plot_roc(label_test, prob, title='', ax=ax)
ax.set_xlabel('1 - Specificity')
ax.set_ylabel('Sensitivity')

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
best_threshold = plot_thresh_metric(ax, y_test, pos_prob, lower=0.1, upper=0.81, n_vals=100)
ax.text(0.71, ax.get_ylim()[1] * 0.9, f'Optimum Threshold = {best_threshold[0]}', fontsize=12, color='b')
print(best_threshold)

if save:
  fig.savefig(args.figdir/f'{args.model}_threshold_guide.pdf', dpi=300)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
plot_thresh_range(ax, y_test, pos_prob, lower=0.1, upper=0.81, n_vals=100)

if save:
  fig.savefig(args.figdir/f'{args.model}_metrics_vary.pdf', dpi=300)

In [ ]:
threshold = 0.31
pred = (pos_prob > threshold).astype(np.int64)
label_preds = [labels[i] for i in pred]
cm = confusion_matrix(y_test, pred)
tn,fp,fn,tp = cm[0][0],cm[0][1],cm[1][0],cm[1][1]
sensitivity = tp/(tp+fn)
specificity = tn/(tn+fp)
ppv = tp/(tp+fp)
npv = tn/(tn+fn)
f1 = (2*ppv*sensitivity)/(ppv+sensitivity)
auroc = roc_auc_score(y_test, pos_prob)

d = {
  'sensitivity': np.round(sensitivity, 3),
  'specificity': np.round(specificity, 3),
  'ppv': np.round(ppv, 3),
  'npv': np.round(npv, 3),
  'f1': np.round(f1, 3),
  'auroc': np.round(auroc, 3),
  'threshold': threshold,
}
metrics = pd.DataFrame(d.values(), index=d.keys(), columns=['Value'])
metrics

In [ ]:
fig, ax = plt.subplots(figsize=(11, 8))
plot_confusion_matrix(label_test, label_preds, x_tick_rotation=45, ax=ax, normalize=True)

In [ ]:
idx2tok = {v: k for k, v in vectorizer.vocabulary_.items()}
fig, ax = plt.subplots(1, 2, figsize=(20, 10))
pos_cloud, neg_cloud = compute_cloud(clf.coef_[0], idx2tok, n_tokens=250)
ax[0].imshow(neg_cloud)
ax[0].axis('off')
ax[0].set_title('Delayed ICU Admission')
ax[1].imshow(pos_cloud)
ax[1].axis('off')
ax[1].set_title('Imminent ICU Admission')

if save:
  fig.savefig(args.figdir/f'wordcloud.pdf', dpi=300)

## Metrics

In [ ]:
with open(args.workdir/f'{args.expt}_preds.pkl', 'rb') as f:
  targs = pickle.load(f)
  probs = pickle.load(f)
  preds = pickle.load(f)

bam = BinaryAvgMetrics(targs, preds, [prob[:, 1] for prob in probs])
bam.get_avg_metrics(defn=True)

In [ ]:
bam.get_avg_metrics(conf=0.95)

In [ ]:
save = False

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
plot_mean_roc(ax, bam.targs, probs)

if save:
  fig.savefig(args.figdir/f'{args.expt}_mean_roc.pdf', dpi=300)

In [ ]:
fig, ax = plt.subplots(figsize=(11, 8))
plot_cm(ax, bam.cm_avg, ['Delayed', 'Imminent'])

if save:
  fig.savefig(args.figdir/f'{args.prefix}_mean_cm.pdf', dpi=300)